In [1]:
import urllib
import json
from bs4 import BeautifulSoup
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
import seaborn as sns
import sys
import time
import networkx as nx
import matplotlib.pyplot as plt
import re
from collections import defaultdict
%matplotlib inline

In [13]:
# brunch data crawling by 셀레니엄
# source reference : http://stackoverflow.com/questions/12519074/scrape-websites-with-infinite-scrolling

# category_list = ['지구한바퀴_세계여행?q=g','그림·웹툰?q=g','시사·이슈?q=g','IT_트렌드?q=g','사진·촬영?q=g','',
#                  '취향저격_영화_리뷰?q=g','오늘은_이런_책?q=g','뮤직_인사이드?q=g','글쓰기_코치?q=g','직장인_현실_조언?q=g'
#                  ,'스타트업_경험담?q=g','육아_이야기?q=g','요리·레시피?q=g','건강·운동?q=g','멘탈_관리_심리_탐구?q=g',
#                  '디자인_스토리?q=g','문화·예술?q=g','건축·설계?q=g','인문학·철학?q=g','쉽게_읽는_역사?q=g','우리집_반려동물?q=g',
#                  '멋진_캘리그래피?q=g','사랑·이별?q=g','감성_에세이?q=g']

category_list = ['지구한바퀴_세계여행?q=g','그림·웹툰?q=g']
idlist = []

def crawlBrunchLink(dir):
    ## html crawling
    chromedriver = 'C:/selenium/chromedriver.exe'
    driver = webdriver.Chrome(chromedriver)
    #driver.implicitly_wait(30)
    base_url = "https://brunch.co.kr/keyword/{dir}".format(dir=dir)
    verificationErrors = []
    accept_next_alert = True
    delay = 0.5
    driver.get(base_url)
    #driver.find_element_by_link_text("All").click()
    htmlsize = 0
    keep_cnt = 0
    for i in range(1,2):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        
        if htmlsize == len(driver.page_source):
            keep_cnt += 1
        else :
            keep_cnt = 0
            htmlsize = len(driver.page_source)
        if keep_cnt > 5 :
            break
            
    html_source = driver.page_source
    ## extract follower, following data
    data = html_source.encode('utf-8')

    soup = BeautifulSoup(data, 'html.parser')
    classes = soup.find_all('a', href=re.compile('/@@'))
#     idlist = []
    for c in classes:
        follwing = c.get('href')
        if follwing is None or len(follwing)==0:
            continue
        idlist.append(follwing[2:])

    driver.close()
    return idlist

In [14]:
writer_list = []

for dir in category_list:
    crawlBrunchLink(dir)
    writer_list.append(idlist)

In [ ]:
# import requests
# from multiprocessing import Pool, Manager
# pool = Pool(processes=4)


# # manager = Manager()
# # res_dict = manager.dict()

# def pool_def(category_list):
#     res_dict = {}
#     tmp_set=[]
#     for i,name in enumerate(category_list):
#         print("%d"%i)
#         res_dict[name] = tmp
#         for text_url in writer_list[i]:
#             tmp.append(tmp_set)
#             tmp_set = []
#             html = requests.get('https://brunch.co.kr/@{text_url}'.format(text_url=text_url))
#             soup = BeautifulSoup(html.text, 'html.parser')
#             ss = soup.find_all('p')
#             tmp_set = []
#             for j in ss:
#                 tmp_set.append(j.text)
#     return res_dict
    
# pool.map(pool_def,category_list)

In [15]:
from multiprocessing import Pool, Manager
import pool_def

if __name__ ==  '__main__': 
    num_processors = 4
    p=Pool(processes = num_processors)
    p.map(pool_def.pool_def ,category_list, writer_list)

TypeError: '<=' not supported between instances of 'list' and 'int'

In [200]:
res_dict['지구한바퀴_세계여행?q=g']

['You can make anythingby writing',
 '- C.S.Lewis -',
 '사파, 힐링여행, 숙소추천\xa0',
 '베트남 여행이 뜨면서 하노이 근교 작은 마을들이 눈길을 끌고 있는데요,',
 '이중 SNS에서 가장 핫한 마을은\xa0자연 그대로의 모습을 간직한 사파!',
 '두 달 전 예약은 필수라는\xa0에포 팜스 하우스 - 사파 리트리트를 소개하겠습니다\xa0:)',
 '',
 '',
 '',
 '에코 팜스 하우스 – 사파 리트리트',
 '',
 '가장 인기 있는 객실은 탁 트인 전망을 고스란히 즐길 수 있는 ‘방갈로’입니다. 인스타그램에서 보신 모든 인생샷이 탄생한 장소예요. 친환경 하우스지만 커피포트, 냉난방 시설, 금고, 멀티탭, wi-fi 등 현대 시설도 잘 준비되어 있어요. 모기장과 우산, 옷걸이는 기본! 함정은 냉장고가 없어요. \xa0간식 챙겨 오셔도 소용없습니다.',
 '',
 '',
 '기본 세면도구와 수건, 슬리퍼, 헤어드라이어 등이 완벽하게 갖춰져 있어요. 샴푸와 바디워시는 향이 너무 좋아서 챙겨 오고 싶을 정도예요. 머릿결이 건조하신 분들은 린스나 컨디셔너만 챙겨주세요. 다만 모든 방갈로가 욕조를 갖추고 있지는 않아요. 예약하시면서 한 번 더 확인해 주세요.',
 '',
 '',
 '아침 식사는 무료예요. 메뉴판을 보시고 주문하시면 됩니다. 무제한으로 가능하세요. 내부 식당에서는 점심, 저녁 식사도 서비스하는데요, 베트남 평균 물가와 비교하면 살짝 비싸지만 사파 최고의 셰프라고 자신합니다. 아무거나 주문해도 다 맛있어요. 추천 메뉴는 돼지고기 요리와 로컬 와인! 역대급 감동을 보장합니다.',
 '',
 '',
 '누군가는 사파를 리틀 스위스라고 부른다네요. 아침에 방갈로에서 눈을 뜨면 실제로 보실 수 있는 전망입니다. 어떠세요? 정원과 산, 강 등 세 가지 옵션 중에서 전망을 선택하실 수 있는데요, 하나를 고르기 어려울 정도로 모두 추천합니다.',
 '',
 '',
 '방갈로는 항상 예약이 꽉 차 있어요. 놓

In [190]:
res_dict.keys()

dict_keys(['지구한바퀴_세계여행?q=g', '그림·웹툰?q=g'])